## Creating an image decoder (Classifying)

In [1]:
import numpy as np
import torch
import os
import cv2

import torch.nn as nn

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using CUDA.")
else:
    device = torch.device("cpu")
    print("Using CPU.")

Using CUDA.


In [3]:
# Import data
folder_path = 'video_frames/'

image_paths = os.listdir(folder_path)
image_paths = [folder_path + img_path for img_path in image_paths]

In [4]:
# Prepare data
y = []
for img in image_paths:
    y.append( cv2.imread(img, cv2.IMREAD_GRAYSCALE).flatten() )

y = torch.tensor( np.array(y) )
X = torch.tensor( np.arange(y.shape[0]) )

In [5]:
train_size = y.shape[0]
print(f"Train size: train_size")

Train size: train_size


### Fully connected NN: 

In [6]:
class FC_Decoder(nn.Module):
    def __init__(self, num_classes= y.shape[0]):
        super(FC_Decoder, self).__init__()

        # Fully connected layers
        self.linear_block = nn.Sequential(
            nn.Linear(1,11),
            nn.ReLU(),
            nn.Linear(11,78),
            nn.ReLU(),
            nn.Linear(78, 536),
            nn.ReLU(),
            nn.Linear(536, 3678),
            nn.ReLU(),
            nn.Linear(3678, 25211),
            nn.ReLU(),
            nn.Linear(25211, 172800)
        )

    def forward(self, x):
        x = self.linear_block(x)
        return x

In [7]:
model = FC_Decoder()
print(model)

FC_Decoder(
  (linear_block): Sequential(
    (0): Linear(in_features=1, out_features=11, bias=True)
    (1): ReLU()
    (2): Linear(in_features=11, out_features=78, bias=True)
    (3): ReLU()
    (4): Linear(in_features=78, out_features=536, bias=True)
    (5): ReLU()
    (6): Linear(in_features=536, out_features=3678, bias=True)
    (7): ReLU()
    (8): Linear(in_features=3678, out_features=25211, bias=True)
    (9): ReLU()
    (10): Linear(in_features=25211, out_features=172800, bias=True)
  )
)


In [8]:
"""
# Hyperparameters
decay_1 = 0.9 # Decay of moving average of gradient
decay_2 = 0.99 # Decay of moving average of squared gradient

lr = 0.0001
weight_decay = 0.00004

lr_decay_rate = 0.98
"""

'\n# Hyperparameters\ndecay_1 = 0.9 # Decay of moving average of gradient\ndecay_2 = 0.99 # Decay of moving average of squared gradient\n\nlr = 0.0001\nweight_decay = 0.00004\n\nlr_decay_rate = 0.98\n'

In [9]:
"""
# Loss function, Optimizer and Scheduler
criterion = torch.nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), 
                              betas = (decay_1, decay_2),
                              lr=lr, 
                              weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=lr_decay_rate)
"""

'\n# Loss function, Optimizer and Scheduler\ncriterion = torch.nn.MSELoss()\noptimizer = torch.optim.AdamW(model.parameters(), \n                              betas = (decay_1, decay_2),\n                              lr=lr, \n                              weight_decay=weight_decay)\nscheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=lr_decay_rate)\n'

In [10]:
# Loss function, Optimizer and Scheduler
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

In [11]:
inputs, labels = X, y
num_epochs = 1000
verbose = True

In [12]:
model.to(device)
inputs.to(device)
labels.to(device)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)

In [13]:
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train() # Set to train mode

    # Zero the gradients
    optimizer.zero_grad()
    
    # Feed-forward
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backprop
    loss.backward()
    optimizer.step()
    
    # Add loss
    batch_loss = loss.item()
    running_loss += batch_loss
    loss_values.append(batch_loss)
    
    # Average loss for the epoch
    avg_loss = running_loss / train_size
 
    # Update the learning rate at the end of each epoch
    scheduler.step()
    
    if verbose == True:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}")
        print(f"Train accuracy: {train_acc}, Validation accuracy: {val_acc}")

    if val_acc > 0.999:
        print("Achieved acceptable accuracy. Stopping early.")
        break

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [16]:
model_device = next(model.parameters()).device
inputs_device = inputs.device
labels_device = labels.device

print(model_device)
print(inputs_device)
print(labels_device)

cuda:0
cpu
cpu


In [17]:
inputs.to(device)

tensor([   0,    1,    2,  ..., 6569, 6570, 6571], device='cuda:0')

In [18]:
inputs_device = inputs.device

In [19]:
print(inputs_device)

cpu


In [20]:
num_params = sum(p.numel() for p in model.parameters())
model_size_mb = num_params * 4 / 1e6

print(f"Number of parameters: {num_params}")
print(f"Model size: {model_size_mb:.2f} MB")

Number of parameters: 4451403257
Model size: 17805.61 MB


### Model is too large, due to using fully connected layers.

In [1]:
del model
del inputs
del labels

torch.cuda.empty_cache()

NameError: name 'model' is not defined